In [ ]:
%pip install -U petals

In [ ]:
from langchain.llms import Petals

In [ ]:
model_name = "../llama-2/llama-2-70b-chat-hf"

llm = Petals(model_name=model_name)

In [ ]:
# REF: https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/model.py#L24
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant.\nYou will try to answer user questions, but don't make up the answer if you don't have the answer.\nYou will complete tasks by following user instructions."
def get_prompt(message: str, chat_history: list[tuple[str, str]] = [],
               system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    texts = [f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    for user_input, response in chat_history:
        texts.append(f'{user_input.strip()} [/INST] {response.strip()} </s><s> [INST] ')
    texts.append(f'{message.strip()} [/INST]')
    return ''.join(texts)

In [ ]:
get_prompt("What is AI?")

In [ ]:
llm(get_prompt("What is AI?"))